# ハイパーパラメーターを調整する

*ハイパーパラメーター* (トレーニングに影響を与えるが、トレーニングデータ自体からは決定できないパラメーター値) を必要とする機械学習アルゴリズムは多数あります。たとえば、ロジスティック回帰モデルをトレーニングする場合、*正規化率*ハイパーパラメーターを使用してモデルのバイアスに対抗できます。畳み込みニューラル ネットワークをトレーニングする場合、*学習率*や*バッチ サイズ*などのハイパーパラメーターを使用して、重みの調整方法とミニバッチで処理されるデータ項目の数をそれぞれ制御できます。ハイパーパラメーター値の選択は、トレーニング済みモデルのパフォーマンスやトレーニングにかかる時間に大きく影響する可能性があります。多くの場合、最適なソリューションを見つけるには複数の組み合わせを試す必要があります。

この場合、2 つのハイパーパラメーターを使用した分類モデルをトレーニングしますが、Azure Machine Learning でトレーニングできるあらゆる種類のモデルに原則が適用されます。

## ワークスペースに接続する

作業を開始するには、ワークスペースに接続します。

> **注**: Azure サブスクリプションでまだ認証済みのセッションを確立していない場合は、リンクをクリックして認証コードを入力し、Azure にサインインして認証するよう指示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存された構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## データを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。次のセルを実行してこのデータセットを作成します (すでに存在する場合は、既存のバージョンが使用されます)

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                        target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                        overwrite=True, # 同じ名前の既存のファイルを置き換える
                        show_progress=True)

    #データストア上のパスから表形式のデータセットを作成する (しばらく時間がかかる場合があります)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 表形式のデータセットを登録する
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## トレーニング スクリプトを準備する

それでは、モデルのトレーニングに使用するトレーニング スクリプト用フォルダーを作成します。

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

ここで、モデルをトレーニングする Python スクリプトを作成します。この例では、*勾配ブースティング* アルゴリズムを使用して分類モデルをトレーニングします。スクリプトには次のものが含まれている必要があります。

- 最適化する各ハイパーパラメーターの引数 (この場合、勾配ブースティング アルゴリズムの学習率と推定器の数)
- 最適化するパフォーマンス メトリックを記録するコード (この場合、AUC と精度の両方を記録するため、どちらかのモデルを最適化することを選択できます)

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# 実験実行コンテキストを取得する
run = Run.get_context()

# スクリプト引数を取得する
parser = argparse.ArgumentParser()

# 入力データセット
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# ハイパーパラメーター
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# args コレクションに引数を追加する
args = parser.parse_args()

# ハイパーパラメーター値を記録する
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# 糖尿病データセットを読み込む
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Estimator 入力からトレーニング データを取得する

# 特徴とラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 指定されたハイパーパラメーターを使用して勾配ブースティング分類モデルをトレーニングする
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 実行出力にモデルを保存する
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## コンピューティングを作成する

ハイパーパラメーター調整では、異なるハイパーパラメーター値を使用して複数のトレーニング反復を実行し、結果として得られたモデルのパフォーマンス メトリックを比較します。これを効率的に行うために、オンデマンドのクラウド コンピューティングを利用してクラスターを作成し、複数のトレーニング反復を同時に実行できるようにします。

次のコードを使用して Azure Machine Learning コンピューティング クラスター (存在しない場合は作成されます) を指定します。

> **重要**: 実行する前に、以下のコードで *your-compute-cluster* をコンピューティング クラスターの名前に変更してください。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。英字、数字、- の文字が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 既存のコンピューティング先を確認する
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # まだ存在しない場合は、作成します
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

## ハイパーパラメーター調整実験を実行する

Azure Machine Learning には、*ハイパードライブ*実験を通じたハイパーパラメーター調整機能が含まれています。これらの実験は、それぞれが異なるハイパーパラメーターの組み合わせで、複数の子の実行を起動します。最適なモデルを作成する実行 (最適化するターゲットのパフォーマンス メトリックの記録によって決定される) を特定し、登録およびデプロイに選択されたトレーニング済みモデルを特定できます。

> **注**: この例では、早期終了ポリシーを指定していません。このようなポリシーは、トレーニング スクリプトが複数のトレーニング反復を実行し、反復ごとにプライマリ メトリックのログを記録する場合にのみ関連します。このアプローチは、複数の*エポック*にわたって深いニューラル ネットワーク モデルをトレーニングする場合に一般的に採用されています。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# 実験用 Python 環境を作成する
sklearn_env = Environment("sklearn-env")

# 必要なパッケージがインストールされていることを確認する (scikit-learn、Azure ML defaults、Azure ML dataprep が必要)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# トレーニング データセットを取得する
diabetes_ds = ws.datasets.get("diabetes dataset")

# スクリプト構成を作成する
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=sklearn_env,
                                compute_target = training_cluster)

# パラメーター値の範囲をサンプリングする
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# ハイパードライブ設定を構成する
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# 実験を実行する
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# 実験の実行時に Notebook の状態を表示する
RunDetails(run).show()
run.wait_for_completion()

上記のウィジェットで実験の実行状態を表示できます。また、[Azure Machine Learning Studio](https://ml.azure.com) で、メインのハイパードライブ実験の実行とその子の実行を表示することもできます。

> **注**: 数値以外は表示できないことを示すメッセージが表示された場合は無視することができます。

## パフォーマンスが最適な実行を決定する

すべての実行が終了したら、指定したパフォーマンス メトリック (この場合は最高の AUC を持つもの) に基づいて最適な実行を見つけることができます。

In [ ]:
# プライマリ メトリックで並べ替えられたすべての子実行を表示する
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# 最適な実行、およびそのメトリックと引数を取得する
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

最適な実行が見つかったので、トレーニングしたモデルを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# 登録済みモデルを一覧表示する
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **詳細情報**: ハイパードライブの詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters)を参照してください。